Load Libraries and Function

In [124]:
import pandas as pd
import numpy as np 
import pymysql as mysql 
import warnings
import matplotlib.pyplot as plt 


Import Datasets

In [125]:
learn = pd.read_csv("learning_loss_covid_database.csv")
demos = pd.read_csv("https://storage.googleapis.com/covid19-open-data/v3/demographics.csv")
epi = pd.read_csv("https://storage.googleapis.com/covid19-open-data/v3/epidemiology.csv")
health = pd.read_csv("https://storage.googleapis.com/covid19-open-data/v3/health.csv")
vacc = pd.read_csv("https://storage.googleapis.com/covid19-open-data/v3/vaccinations.csv")
gdp = pd.read_csv("GDPForecast.csv")


Examine DataFrames Info: Shape, Datatypes, and Missing Values

In [126]:
learn.info()
demos.info() 
epi.info()
health.info() 
vacc.info()
gdp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41 entries, 0 to 40
Data columns (total 24 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   country      41 non-null     object 
 1   loss         41 non-null     float64
 2   weeks        41 non-null     int64  
 3   deaths       41 non-null     float64
 4   gdp          41 non-null     float64
 5   private      41 non-null     float64
 6   internet     41 non-null     int64  
 7   hlo          41 non-null     float64
 8   vac          41 non-null     float64
 9   stringency   41 non-null     float64
 10  union        41 non-null     float64
 11  polity       41 non-null     int64  
 12  loggdp       41 non-null     float64
 13  logweeks     39 non-null     float64
 14  weeks2       41 non-null     int64  
 15  pop          41 non-null     int64  
 16  schooling    41 non-null     float64
 17  wbcode       41 non-null     object 
 18  region_code  41 non-null     object 
 19  hlo25     

View Dataframes and Make Note of Redundant Variables within Datasets and Between Datasets

In [127]:
# Redundant information: deaths, gdp, vac, loggdp, logweeks, weeks2, pop, hlo25, covid25
# Update column 1 name to "location_key"
learn.head()

,country,loss,weeks,deaths,gdp,private,internet,hlo,vac,stringency,...,weeks2,pop,schooling,wbcode,region_code,hlo25,oecd,covid,high,covid25
0,Argentina,0.44,39,288.00,8585.69,26.08010,86,457.418,65.0681,67.0396,...,1521,45000000,9.51,ARG,Latin America and the Caribbean,18.2967,0,413.418,0,16.5367
1,Australia,0.01,8,60.37,51680.30,29.53870,90,544.360,64.0000,61.0000,...,64,26000000,11.54,AUS,Advanced Economies,21.7744,1,543.360,1,21.7344
2,Bangladesh,0.28,63,17.84,2270.35,23.90350,25,340.088,17.0000,70.0000,...,3969,170000000,5.91,BGD,South Asia,13.6035,0,312.088,0,12.4835
3,Belgium,0.23,9,284.67,45189.40,54.17840,92,340.088,87.0000,50.0000,...,81,11000000,10.69,BEL,Advanced Economies,13.6035,1,317.088,1,12.6835
4,Botswana,0.29,11,118.60,6348.83,6.93809,64,417.656,12.0000,50.0000,...,121,2300000,9.55,BWA,Sub-Saharan Africa,16.7062,0,388.656,0,15.5462


In [128]:
# Redundant information: Shows total country data, and then breaks down some countries by region or province. Need to reduce to whole country data only.
demos.head()

,location_key,population,population_male,population_female,population_rural,population_urban,population_largest_city,population_clustered,population_density,human_development_index,population_age_00_09,population_age_10_19,population_age_20_29,population_age_30_39,population_age_40_49,population_age_50_59,population_age_60_69,population_age_70_79,population_age_80_and_older
0,AD,77265.0,58625.0,55581.0,9269.0,67873.0,NaN,NaN,164.394,0.858,9370.0,12022.0,10727.0,12394.0,21001.0,20720.0,14433.0,8657.0,4881.0
1,AE,9890400.0,6836349.0,3054051.0,1290785.0,8479744.0,2833079.0,5914068.0,118.306,0.863,1011713.0,842991.0,2149343.0,3169314.0,1608109.0,797913.0,242707.0,55884.0,12426.0
2,AF,38928341.0,19976265.0,18952076.0,28244481.0,9797273.0,4114030.0,4114030.0,59.627,0.498,11088732.0,9821559.0,7035871.0,4534646.0,2963459.0,1840198.0,1057496.0,480455.0,105925.0
3,AF_BAL,1509183.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,AF_BAM,495557.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [129]:
# Needs to be reduced down to largest cummulative total for each country
epi.head()

,date,location_key,new_confirmed,new_deceased,new_recovered,new_tested,cumulative_confirmed,cumulative_deceased,cumulative_recovered,cumulative_tested
0,2020-01-01,AD,0.0,0.0,NaN,NaN,0.0,0.0,NaN,NaN
1,2020-01-02,AD,0.0,0.0,NaN,NaN,0.0,0.0,NaN,NaN
2,2020-01-03,AD,0.0,0.0,NaN,NaN,0.0,0.0,NaN,NaN
3,2020-01-04,AD,0.0,0.0,NaN,NaN,0.0,0.0,NaN,NaN
4,2020-01-05,AD,0.0,0.0,NaN,NaN,0.0,0.0,NaN,NaN


In [130]:
# Need to reduce down to cummulative country totals only. Some have additional region information.
health.head()

,location_key,life_expectancy,smoking_prevalence,diabetes_prevalence,infant_mortality_rate,adult_male_mortality_rate,adult_female_mortality_rate,pollution_mortality_rate,comorbidity_mortality_rate,hospital_beds_per_1000,nurses_per_1000,physicians_per_1000,health_expenditure_usd,out_of_pocket_health_expenditure_usd
0,AD,NaN,33.5,7.7,2.7,NaN,NaN,NaN,NaN,NaN,4.0128,3.3333,4040.786621,1688.121460
1,AE,77.814,28.9,16.3,6.5,69.555,44.863,54.7,16.8,NaN,5.7271,2.5278,1357.017456,256.034485
2,AF,64.486,NaN,9.2,47.9,237.554,192.532,211.1,29.8,0.5,0.1755,0.2782,67.122650,50.665913
3,AG,76.885,NaN,13.1,5.0,126.917,83.136,29.9,22.6,NaN,4.5171,2.9560,673.859680,235.749039
4,AL,78.900,28.7,9.0,7.8,93.315,49.486,68.0,17.0,NaN,3.6495,1.2164,NaN,NaN


In [131]:
# Times series data needs to be reduced to maximum cummulative numbers.
vacc.head()

,date,location_key,new_persons_vaccinated,cumulative_persons_vaccinated,new_persons_fully_vaccinated,cumulative_persons_fully_vaccinated,new_vaccine_doses_administered,cumulative_vaccine_doses_administered,new_persons_vaccinated_pfizer,cumulative_persons_vaccinated_pfizer,...,new_persons_fully_vaccinated_janssen,cumulative_persons_fully_vaccinated_janssen,new_vaccine_doses_administered_janssen,cumulative_vaccine_doses_administered_janssen,new_persons_vaccinated_sinovac,total_persons_vaccinated_sinovac,new_persons_fully_vaccinated_sinovac,total_persons_fully_vaccinated_sinovac,new_vaccine_doses_administered_sinovac,total_vaccine_doses_administered_sinovac
0,2021-01-25,AD,NaN,576.0,NaN,NaN,NaN,576.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2021-02-01,AD,460.0,1036.0,NaN,NaN,460.0,1036.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2021-02-10,AD,255.0,1291.0,NaN,NaN,255.0,1291.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2021-02-12,AD,331.0,1622.0,NaN,NaN,331.0,1622.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2021-02-19,AD,519.0,2141.0,NaN,NaN,519.0,2141.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [132]:
# Needs to be trimmed to years of pandemic and forecast only
# Need to update Column 1 name to "location_key"
gdp.head()

,Real GDP growth (Annual percent change),1980,1981,1982,1983,1984,1985,1986,1987,1988,...,2019,2020,2021,2022,2023,2024,2025,2026,2027,2028
0,Afghanistan,no data,no data,no data,no data,no data,no data,no data,no data,no data,...,3.9,-2.4,-20.7,no data,no data,no data,no data,no data,no data,no data
1,Albania,2.7,5.7,2.9,1.1,2,-1.5,5.6,-0.8,-1.4,...,2.1,-3.3,8.9,4.8,3.6,3.3,3.4,3.5,3.5,3.5
2,Algeria,-5.4,3,6.4,5.4,5.6,5.6,-0.2,-0.7,-1.9,...,1,-5.1,3.4,3.2,3.8,3.1,2.5,1.9,1.7,1.7
3,Andorra,no data,no data,no data,no data,no data,no data,no data,no data,no data,...,2,-11.2,8.3,8.8,2.1,1.5,1.5,1.5,1.5,1.5
4,Angola,2.4,-4.4,0,4.2,6,3.5,2.9,4.1,6.1,...,-0.7,-5.6,1.2,3,1.3,3.3,3.4,3.5,3.5,3.6


Updating Dataframes to Reflect only Necessary Data

Learning data needs some columns dropped.

In [136]:
learn = learn.drop(columns = ['deaths', 'gdp', 'vac', 'loggdp', 'logweeks', 'weeks2', 'pop', 'hlo25', 'covid25'])

# Update column 1 name
learn.rename(columns={'country': 'location_key'}, inplace=True)

learn.head()

,location_key,loss,weeks,private,internet,hlo,stringency,union,polity,schooling,wbcode,region_code,oecd,covid,high
0,Argentina,0.44,39,26.08010,86,457.418,67.0396,27.7,9,9.51,ARG,Latin America and the Caribbean,0,413.418,0
1,Australia,0.01,8,29.53870,90,544.360,61.0000,13.7,10,11.54,AUS,Advanced Economies,1,543.360,1
2,Bangladesh,0.28,63,23.90350,25,340.088,70.0000,11.9,-6,5.91,BGD,South Asia,0,312.088,0
3,Belgium,0.23,9,54.17840,92,340.088,50.0000,49.1,8,10.69,BEL,Advanced Economies,1,317.088,1
4,Botswana,0.29,11,6.93809,64,417.656,50.0000,30.0,8,9.55,BWA,Sub-Saharan Africa,0,388.656,0


Demographics data needs to be reduced to full country stats only. Delete regional data.

In [137]:
demos = demos[demos['location_key'].str.len() == 2]

In [138]:
demos.head()

,location_key,population,population_male,population_female,population_rural,population_urban,population_largest_city,population_clustered,population_density,human_development_index,population_age_00_09,population_age_10_19,population_age_20_29,population_age_30_39,population_age_40_49,population_age_50_59,population_age_60_69,population_age_70_79,population_age_80_and_older
0,AD,77265.0,58625.0,55581.0,9269.0,67873.0,NaN,NaN,164.394,0.858,9370.0,12022.0,10727.0,12394.0,21001.0,20720.0,14433.0,8657.0,4881.0
1,AE,9890400.0,6836349.0,3054051.0,1290785.0,8479744.0,2833079.0,5914068.0,118.306,0.863,1011713.0,842991.0,2149343.0,3169314.0,1608109.0,797913.0,242707.0,55884.0,12426.0
2,AF,38928341.0,19976265.0,18952076.0,28244481.0,9797273.0,4114030.0,4114030.0,59.627,0.498,11088732.0,9821559.0,7035871.0,4534646.0,2963459.0,1840198.0,1057496.0,480455.0,105925.0
37,AG,97928.0,47269.0,50659.0,73318.0,23800.0,NaN,NaN,222.564,0.780,14583.0,13867.0,14894.0,14094.0,13860.0,12832.0,8043.0,4101.0,1654.0
38,AI,15002.0,7151.0,7971.0,NaN,NaN,NaN,NaN,166.689,NaN,2196.0,2146.0,2005.0,2081.0,2187.0,2214.0,1351.0,641.0,301.0


In [139]:
demos['location_key'].value_counts()

AD    1
MY    1
NC    1
NE    1
NF    1
     ..
GR    1
GS    1
GT    1
GU    1
ZW    1
Name: location_key, Length: 245, dtype: int64

COVID Epidemiology data needs to be reduced to final pandemic numbers, i.e., the maximum cummulative totals.

In [140]:
# Drop "new" columns and retain only cummulative columns
epi = epi.drop(columns = ['new_confirmed', 'new_deceased', 'new_recovered', 'new_tested'])

In [141]:
# Drop regional country information and retain only primary country codes
epi = epi[epi['location_key'].str.len() == 2]

In [142]:
# Find max value, or most recent date for each country
epi = epi.groupby(by = 'location_key', as_index = False).max()
epi.head()

,location_key,date,cumulative_confirmed,cumulative_deceased,cumulative_recovered,cumulative_tested
0,AD,2022-09-13,46147.0,155.0,NaN,300307.0
1,AE,2022-09-13,1021191.0,2342.0,NaN,168522672.0
2,AF,2022-09-13,196663.0,7791.0,NaN,994894.0
3,AG,2022-09-13,9008.0,145.0,NaN,16700.0
4,AI,2022-09-13,3856.0,12.0,NaN,51382.0


In [143]:
epi.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 232 entries, 0 to 231
Data columns (total 6 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   location_key          232 non-null    object 
 1   date                  232 non-null    object 
 2   cumulative_confirmed  231 non-null    float64
 3   cumulative_deceased   231 non-null    float64
 4   cumulative_recovered  20 non-null     float64
 5   cumulative_tested     185 non-null    float64
dtypes: float64(4), object(2)
memory usage: 11.0+ KB


In [144]:
epi['location_key'].value_counts()

AD    1
PE    1
NC    1
NE    1
NG    1
     ..
GT    1
GU    1
GW    1
GY    1
ZW    1
Name: location_key, Length: 232, dtype: int64

Health data needs to be reduced to primary country ID code only. Remove regional country information.

In [145]:
health = health[health['location_key'].str.len() == 2]
health.head()

,location_key,life_expectancy,smoking_prevalence,diabetes_prevalence,infant_mortality_rate,adult_male_mortality_rate,adult_female_mortality_rate,pollution_mortality_rate,comorbidity_mortality_rate,hospital_beds_per_1000,nurses_per_1000,physicians_per_1000,health_expenditure_usd,out_of_pocket_health_expenditure_usd
0,AD,NaN,33.5,7.7,2.7,NaN,NaN,NaN,NaN,NaN,4.0128,3.3333,4040.786621,1688.121460
1,AE,77.814,28.9,16.3,6.5,69.555,44.863,54.7,16.8,NaN,5.7271,2.5278,1357.017456,256.034485
2,AF,64.486,NaN,9.2,47.9,237.554,192.532,211.1,29.8,0.5,0.1755,0.2782,67.122650,50.665913
3,AG,76.885,NaN,13.1,5.0,126.917,83.136,29.9,22.6,NaN,4.5171,2.9560,673.859680,235.749039
4,AL,78.900,28.7,9.0,7.8,93.315,49.486,68.0,17.0,NaN,3.6495,1.2164,NaN,NaN


In [146]:
health.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 209 entries, 0 to 3503
Data columns (total 14 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   location_key                          209 non-null    object 
 1   life_expectancy                       204 non-null    float64
 2   smoking_prevalence                    145 non-null    float64
 3   diabetes_prevalence                   208 non-null    float64
 4   infant_mortality_rate                 192 non-null    float64
 5   adult_male_mortality_rate             188 non-null    float64
 6   adult_female_mortality_rate           188 non-null    float64
 7   pollution_mortality_rate              182 non-null    float64
 8   comorbidity_mortality_rate            182 non-null    float64
 9   hospital_beds_per_1000                25 non-null     float64
 10  nurses_per_1000                       179 non-null    float64
 11  physicians_per_100

In [ ]:
health['location_key'].value_counts()

Vaccine information needs to be reduced to cummulative numbers for each primary country code.

In [147]:
vacc = vacc[vacc['location_key'].str.len() == 2]
vacc.head()

,date,location_key,new_persons_vaccinated,cumulative_persons_vaccinated,new_persons_fully_vaccinated,cumulative_persons_fully_vaccinated,new_vaccine_doses_administered,cumulative_vaccine_doses_administered,new_persons_vaccinated_pfizer,cumulative_persons_vaccinated_pfizer,...,new_persons_fully_vaccinated_janssen,cumulative_persons_fully_vaccinated_janssen,new_vaccine_doses_administered_janssen,cumulative_vaccine_doses_administered_janssen,new_persons_vaccinated_sinovac,total_persons_vaccinated_sinovac,new_persons_fully_vaccinated_sinovac,total_persons_fully_vaccinated_sinovac,new_vaccine_doses_administered_sinovac,total_vaccine_doses_administered_sinovac
0,2021-01-25,AD,NaN,576.0,NaN,NaN,NaN,576.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2021-02-01,AD,460.0,1036.0,NaN,NaN,460.0,1036.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2021-02-10,AD,255.0,1291.0,NaN,NaN,255.0,1291.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2021-02-12,AD,331.0,1622.0,NaN,NaN,331.0,1622.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2021-02-19,AD,519.0,2141.0,NaN,NaN,519.0,2141.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [148]:
vacc = vacc.groupby(by = 'location_key', as_index = False).max()
vacc.head()

,location_key,date,new_persons_vaccinated,cumulative_persons_vaccinated,new_persons_fully_vaccinated,cumulative_persons_fully_vaccinated,new_vaccine_doses_administered,cumulative_vaccine_doses_administered,new_persons_vaccinated_pfizer,cumulative_persons_vaccinated_pfizer,...,new_persons_fully_vaccinated_janssen,cumulative_persons_fully_vaccinated_janssen,new_vaccine_doses_administered_janssen,cumulative_vaccine_doses_administered_janssen,new_persons_vaccinated_sinovac,total_persons_vaccinated_sinovac,new_persons_fully_vaccinated_sinovac,total_persons_fully_vaccinated_sinovac,new_vaccine_doses_administered_sinovac,total_vaccine_doses_administered_sinovac
0,AD,2022-09-04,11952.0,57893.0,8182.0,53478.0,18175.0,154195.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,AE,2022-06-20,2643847.0,9991089.0,2493428.0,9792266.0,304988.0,24922054.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,AF,2022-09-12,2701572.0,10827184.0,2757865.0,10105500.0,1782255.0,11734635.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,AG,2022-08-06,24164.0,64091.0,4469.0,62031.0,24164.0,128267.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,AI,2022-08-19,2696.0,10846.0,3641.0,10314.0,6337.0,24102.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [149]:
# Drop redundant columns
vacc = vacc.loc[:,~vacc.columns.str.contains('^new', case=False)]
vacc.head()

,location_key,date,cumulative_persons_vaccinated,cumulative_persons_fully_vaccinated,cumulative_vaccine_doses_administered,cumulative_persons_vaccinated_pfizer,cumulative_persons_fully_vaccinated_pfizer,cumulative_vaccine_doses_administered_pfizer,cumulative_persons_vaccinated_moderna,cumulative_persons_fully_vaccinated_moderna,cumulative_vaccine_doses_administered_moderna,cumulative_persons_vaccinated_janssen,cumulative_persons_fully_vaccinated_janssen,cumulative_vaccine_doses_administered_janssen,total_persons_vaccinated_sinovac,total_persons_fully_vaccinated_sinovac,total_vaccine_doses_administered_sinovac
0,AD,2022-09-04,57893.0,53478.0,154195.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,AE,2022-06-20,9991089.0,9792266.0,24922054.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,AF,2022-09-12,10827184.0,10105500.0,11734635.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,AG,2022-08-06,64091.0,62031.0,128267.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,AI,2022-08-19,10846.0,10314.0,24102.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [150]:
vacc = vacc.iloc[: , [0,2,3]]
vacc.head()

,location_key,cumulative_persons_vaccinated,cumulative_persons_fully_vaccinated
0,AD,57893.0,53478.0
1,AE,9991089.0,9792266.0
2,AF,10827184.0,10105500.0
3,AG,64091.0,62031.0
4,AI,10846.0,10314.0


GDP Information Needs to be reduced to Pandemic Years and Future Forecasts Only: 2019 onward

In [155]:
# Grab desired years
gdp = gdp.iloc[: , [0] + list(range(40, 50))]

# Update column 1 name
gdp.rename(columns={'Real GDP growth (Annual percent change)': 'location_key'}, inplace=True)

# View
gdp.head()

,location_key,2019,2020,2021,2022,2023,2024,2025,2026,2027,2028
0,Afghanistan,3.9,-2.4,-20.7,no data,no data,no data,no data,no data,no data,no data
1,Albania,2.1,-3.3,8.9,4.8,3.6,3.3,3.4,3.5,3.5,3.5
2,Algeria,1,-5.1,3.4,3.2,3.8,3.1,2.5,1.9,1.7,1.7
3,Andorra,2,-11.2,8.3,8.8,2.1,1.5,1.5,1.5,1.5,1.5
4,Angola,-0.7,-5.6,1.2,3,1.3,3.3,3.4,3.5,3.5,3.6


Assessing Country IDs:
Examine the number of countries available in each dataset and the format of the country IDs. Some are abbreviated and some are spelled out. 

In [159]:
# String names for countries
# 41 countries
learn['location_key'].value_counts().shape


(41,)

In [160]:
# Abbreviations for countries
# 245 countries
demos['location_key'].value_counts().shape


(245,)

In [161]:
# Abbreviations for countries
# 232 countries
epi['location_key'].value_counts().shape


(232,)

In [162]:
# Abbreviations for countries
# 209 countries
health['location_key'].value_counts().shape


(209,)

In [163]:
# Abbreviations for countries
# 218 countries
vacc['location_key'].value_counts().shape


(218,)

In [165]:
# Country names as strings
# 228 countries
gdp['location_key'].value_counts().shape

(228,)

GDP and learning loss need to have a column added for country abbreviation.

All other datasets need to have a column added for country string name. 

A Primary Table needs to be made with all country names corresponding to country abbreviation IDs.
